# Домашнее задание № 10. Машинный перевод

# Задание 1 (6 баллов + 2 доп балла).

## Инструкция



Нужно обучить трансформер на том же корпусе но в другую сторону - с русского на английский.
Можно использовать как основу первый или второй способ реализации (с MultiheadAttention или с nn.Transformer). Подберите несколько тестовых примеров для проверки обучения на каждой эпохе.

Параметры ниже точно работают в колабе и модель обучается достаточно быстро. Попробуйте их немного увеличить (batch size возможно придется наоборот уменьшить). Обучайте модель хотя бы 5 эпох, а желательно больше, чтобы тестовые примеры начали переводиться более менее адекватно.

После обучения возьмите хотя бы 100 примером из тестовой части параллельного корпуса и переведите их. Оцените качество переводов с помощью метрики BLEU (пример использования ниже)
Найдите лучшие (как минимум 5) переводы согласно этой метрике и проверьте действительно ли они хорошие. Если все переводы нулевые, то пообучайте модель подольше.

Чтобы получить 2 доп балла вам нужно будет придумать как оптимизировать функцию translate. Сейчас она работает только с одним текстом - это не эффективно. Можно генерировать переводы сразу для нескольких текстов (батча). Главная сложность с таким подходом состоит в том, что генерируемые тексты будут заканчиваться в разное время и нужно сделать столько итераций, сколько нужно для завершения всех текстов (т.е. условие на то, что последний токен не равен [EOS] в текущем коде не сработает).
ВАЖНО - недостаточно просто изменить входной аргумент с text на texts и добавить еще один цикл по texts! Сама модель должна вызываться на нескольких текстах! Функция с batch prediction должна работать быстрее, поэтому переведите всю тестовую выборку и оцените качество BLEU на всех данных.

In [ ]:
# пример использования BLEU
# обратите внимание что текты должны быть токенизированы
import nltk

hypothesis = ['It', 'is', 'a', 'cat', 'at', 'room'] # замените на перевод вашей модели
reference = ['It', 'is', 'a', 'cat', 'inside', 'the', 'room'] # замените на эталонный перевод

BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis, auto_reweigh=True)
print(BLEUscore)

0.4548019047027907


In [ ]:
# параметры которые работают в колабе
embed_dim = 32
num_heads = 4
ff_dim = embed_dim*2
num_layers = 2
batch_size = 400

## Импорты

In [1]:
# В колабе установите torchtune и torchao, чтобы семинарская тетрадка работала
!pip install torchtune torchao

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 910.7/910.7 kB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 120.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 115.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.9 MB/s eta 0:00:00
   ━━━

In [87]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data

from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer
from tokenizers import decoders

import os
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from collections import Counter

from torchtune.modules import RotaryPositionalEmbeddings
from torch.nn import Transformer
import matplotlib.pyplot as plt
%matplotlib inline

from timeit import default_timer as timer
from time import time
from tqdm import tqdm
import nltk

## Данные

In [11]:
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.ru
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.en

!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-test.ru
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-test.en

--2025-04-23 12:10:29--  https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.ru
Resolving data.statmt.org (data.statmt.org)... 129.215.32.28
Connecting to data.statmt.org (data.statmt.org)|129.215.32.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 121340806 (116M)
Saving to: ‘opus.en-ru-train.ru.2’

opus.en-ru-train.ru 100%[===================>] 115.72M  20.8MB/s    in 7.0s    

2025-04-23 12:10:37 (16.5 MB/s) - ‘opus.en-ru-train.ru.2’ saved [121340806/121340806]

--2025-04-23 12:10:37--  https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.en
Resolving data.statmt.org (data.statmt.org)... 129.215.32.28
Connecting to data.statmt.org (data.statmt.org)|129.215.32.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67760131 (65M)
Saving to: ‘opus.en-ru-train.en.2’

opus.en-ru-train.en 100%[===================>]  64.62M  6.76MB/s    in 20s     

2025-04-23 12:10:58 (3.21 MB/s) - ‘opus

In [12]:
# в русскоязычных данных есть \xa0 вместо пробелов, он может некорректно обрабатываться токенизатором
text = open("opus.en-ru-train.ru").read().replace("\xa0", " ")
f = open("opus.en-ru-train.ru", "w")
f.write(text)
f.close()

In [13]:
en_sents = open("opus.en-ru-train.en").read().splitlines()
ru_sents = open("opus.en-ru-train.ru").read().splitlines()

print(len(en_sents))
print(len(ru_sents))

1000000
1000000


In [16]:
# сократим кол-во примеров в пять раз для экономии вычислений

en_sents = en_sents[:len(en_sents) // 5]
ru_sents = ru_sents[:len(ru_sents) // 5]

print(len(en_sents))
print(len(ru_sents))

200000
200000


In [17]:
list(zip(en_sents[:10], ru_sents[:10]))

[("Yeah, that's not exactly...", 'Да, но не совсем...'),
 ('!', '!'),
 ('The schedule below is tentative; up-to-date information can be obtained at www.un.org/News/ossg/conf.htm.',
  'Приводимое ниже расписание является предварительным; с самой последней информацией можно ознакомиться в Интернете по адресу www.un.org/News/ossg/conf.htm.'),
 ('But for now,',
  'Но сейчас ...я вверяю вам удостовериться, что шотландцы приуменьшат'),
 ("He'd been out there a few weeks or so.",
  'Они тусовались там несколько недель.'),
 ("It'll make you feel better.", 'Вам станет лучше.'),
 ('Come in!', 'Войдите.'),
 ('Do the math.', 'Давай, догадывайся.'),
 ('- Jenna?', '- Дженна?'),
 ('My cheekbones and beckoning pelvis already have a certain "hello sailor" quality to them.',
  'Мои скулы и манящий таз уже им подают сигнал "Привет, Матрос"')]

## Токенайзер

In [18]:
tokenizer_ru = Tokenizer(BPE())
tokenizer_ru.pre_tokenizer = Whitespace()

# в encoder нам не нужно обозначать начало и конец поэтому единственный доп токен это паддинг
trainer_ru = BpeTrainer(special_tokens=["[PAD]"], end_of_word_suffix='</w>')
tokenizer_ru.train(files=["opus.en-ru-train.ru"], trainer=trainer_ru)

tokenizer_en = Tokenizer(BPE())
tokenizer_en.pre_tokenizer = Whitespace()

# в декодере добавим теги начала и конца для корректной генерации
trainer_en = BpeTrainer(special_tokens=["[PAD]", "[BOS]", "[EOS]"], end_of_word_suffix='</w>')
tokenizer_en.train(files=["opus.en-ru-train.en"], trainer=trainer_en)

In [19]:
tokenizer_ru.decoder = decoders.BPEDecoder()
tokenizer_en.decoder = decoders.BPEDecoder()

In [20]:
# раскоментируйте эту ячейку при обучении токенизатора
# а потом снова закоментируйте чтобы при перезапуске не перезаписать токенизаторы

tokenizer_ru.save('tokenizer_ru')
tokenizer_en.save('tokenizer_en')

In [21]:
tokenizer_ru = Tokenizer.from_file("tokenizer_ru")
tokenizer_en = Tokenizer.from_file("tokenizer_en")

In [22]:
tokenizer_ru

Tokenizer(version="1.0", truncation=None, padding=None, added_tokens=[{"id":0, "content":"[PAD]", "single_word":False, "lstrip":False, "rstrip":False, "normalized":False, "special":True}], normalizer=None, pre_tokenizer=Whitespace(), post_processor=None, decoder=BPEDecoder(suffix="</w>"), model=BPE(dropout=None, unk_token=None, continuing_subword_prefix=None, end_of_word_suffix="</w>", fuse_unk=False, byte_fallback=False, ignore_merges=False, vocab={"[PAD]":0, "!":1, """:2, "#":3, "$":4, "%":5, "&":6, "'":7, "(":8, ")":9, "*":10, "+":11, ",":12, "-":13, ".":14, "/":15, "0":16, "1":17, "2":18, "3":19, "4":20, "5":21, "6":22, "7":23, "8":24, "9":25, ":":26, ";":27, "<":28, "=":29, ">":30, "?":31, "@":32, "A":33, "B":34, "C":35, "D":36, "E":37, "F":38, "G":39, "H":40, "I":41, "J":42, "K":43, "L":44, "M":45, "N":46, "O":47, "P":48, "Q":49, "R":50, "S":51, "T":52, "U":53, "V":54, "W":55, "X":56, "Y":57, "Z":58, "[":59, "\":60, "]":61, "^":62, "_":63, "`":64, "a":65, "b":66, "c":67, "d":68, 

## Препроцессинг

In [31]:
def encode(text, tokenizer, max_len, encoder=False):
    if encoder:
        return tokenizer.encode(text).ids[:max_len]
    else:
        return [tokenizer.token_to_id('[BOS]')] + \
        tokenizer.encode(text).ids[:max_len] + [tokenizer.token_to_id('[EOS]')]

In [53]:
# важно следить чтобы индекс паддинга совпадал в токенизаторе с value в pad_sequences
# у нас это в любом случае ноль но лучше safe than sorry
PAD_IDX = tokenizer_en.token_to_id('[PAD]')
PAD_IDX

0

In [29]:
max_len_en, max_len_ru = 47, 48

In [60]:
X_en = [encode(t, tokenizer_en, max_len_en) for t in en_sents]
X_ru = [encode(t, tokenizer_ru, max_len_ru, encoder=True) for t in ru_sents]

In [61]:
len(X_en), len(X_ru)

(200000, 200000)

In [62]:
X_ru[0]

[2797, 1690, 2432, 2421, 5615, 2535]

## Датасет

In [55]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, texts_ru, texts_en):
        self.texts_en = [torch.LongTensor(sent) for sent in texts_en]
        self.texts_en = torch.nn.utils.rnn.pad_sequence(self.texts_en, batch_first=True, padding_value=PAD_IDX)

        self.texts_ru = [torch.LongTensor(sent) for sent in texts_ru]
        self.texts_ru = torch.nn.utils.rnn.pad_sequence(self.texts_ru, batch_first=True, padding_value=PAD_IDX)

        self.length = len(texts_ru)

    def __len__(self):
        return self.length

    def __getitem__(self, index):

        ids_en = self.texts_en[index]
        ids_ru = self.texts_ru[index]

        return ids_ru, ids_en

In [63]:
X_ru_train, X_ru_valid, X_en_train, X_en_valid, = \
train_test_split(X_ru, X_en, test_size=0.05)

In [64]:
print(*X_ru_train[0])

3021 1678 3551 1678 4279 1690 2962 1661 3260 3717 1760 1639


In [72]:
training_set = Dataset(X_ru_train, X_en_train)
valid_set = Dataset(X_ru_valid, X_en_valid)

## Определения для обучения

In [74]:
class TransformerEncoderDecoder(nn.Module): # из коробки
    def __init__(self, vocab_size_enc, vocab_size_dec, embed_dim, num_heads, ff_dim, num_layers, dropout=0.1):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.embedding_enc = nn.Embedding(vocab_size_enc, embed_dim)
        self.embedding_dec = nn.Embedding(vocab_size_dec, embed_dim)
        self.positional_encoding = RotaryPositionalEmbeddings(embed_dim // num_heads, max_seq_len=128)

        self.transformer = Transformer(
            d_model=embed_dim,
            nhead=num_heads,
            num_encoder_layers=num_layers,
            num_decoder_layers=num_layers,
            dim_feedforward=ff_dim,
            dropout=dropout,
            batch_first=True
        )

        self.output_layer = nn.Linear(embed_dim, vocab_size_dec)

    def forward(self, src, tgt, src_key_padding_mask=None, tgt_key_padding_mask=None):

        src_embedded = self.embedding_enc(src)
        B,S,E = src_embedded.shape
        src_embedded = self.positional_encoding(src_embedded.view(B,S,self.num_heads, E//self.num_heads)).view(B,S,E)

        tgt_embedded = self.embedding_dec(tgt)
        B,S,E = tgt_embedded.shape
        tgt_embedded = self.positional_encoding(tgt_embedded.view(B,S,self.num_heads, E//self.num_heads)).view(B,S,E)


        tgt_mask = (~torch.tril(torch.ones((S, S), dtype=torch.bool))).to(DEVICE)

        encoder_output = self.transformer.encoder(
            src_embedded,
            src_key_padding_mask=src_key_padding_mask
        )

        decoder_output = self.transformer.decoder(
            tgt_embedded,
            encoder_output,
            tgt_mask=tgt_mask,
            tgt_key_padding_mask=tgt_key_padding_mask,
            memory_key_padding_mask=src_key_padding_mask
        )

        output = self.output_layer(decoder_output)
        return output

In [86]:
def train(model, iterator, optimizer, criterion, scheduler, run=None, print_every=200):

    epoch_loss = []
    ac = []

    model.train()

    for i, (texts_ru, texts_en) in enumerate(iterator):
        texts_en = texts_en.to(DEVICE)
        texts_ru = texts_ru.to(DEVICE)
        texts_en_input = texts_en[:,:-1].to(DEVICE)
        texts_en_out = texts_en[:, 1:].to(DEVICE)
        src_padding_mask = (texts_ru == PAD_IDX).to(DEVICE)
        tgt_padding_mask = (texts_en_input == PAD_IDX).to(DEVICE)
        logits = model(texts_ru, texts_en_input, src_padding_mask, tgt_padding_mask)
        optimizer.zero_grad()
        B,S,C = logits.shape
        loss = loss_fn(logits.reshape(B*S, C), texts_en_out.reshape(B*S))
        loss.backward()
        optimizer.step()
        scheduler.step()
        epoch_loss.append(loss.item())

        if not (i+1) % print_every:
            print(f'Loss: {np.mean(epoch_loss)};')
        if run is not None:
            run.log({"loss": loss.item()})

    return np.mean(epoch_loss)

In [76]:
def evaluate(model, iterator, criterion, run=None):

    epoch_loss = []
    epoch_f1 = []

    model.eval()
    with torch.no_grad():
        for i, (texts_ru, texts_en) in enumerate(iterator):
            texts_en = texts_en.to(DEVICE)
            texts_ru = texts_ru.to(DEVICE)
            texts_en_input = texts_en[:,:-1].to(DEVICE)
            texts_en_out = texts_en[:, 1:].to(DEVICE)
            src_padding_mask = (texts_ru == PAD_IDX).to(DEVICE)
            tgt_padding_mask = (texts_en_input == PAD_IDX).to(DEVICE)

            logits = model(texts_ru, texts_en_input, src_padding_mask, tgt_padding_mask)

            B,S,C = logits.shape
            loss = loss_fn(logits.reshape(B*S, C), texts_en_out.reshape(B*S))
            epoch_loss.append(loss.item())
            if run is not None:
                run.log({"val_loss": loss.item()})

    return np.mean(epoch_loss)

In [90]:
# Дополнительная функция чтобы сгенерировать перевод с нуля для текста,
# чтобы мониторить качество

@torch.no_grad
def translate(text):


    input_ids = tokenizer_ru.encode(text).ids[:max_len_ru]
    output_ids = [tokenizer_en.token_to_id('[BOS]')]

    input_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(input_ids)], \
                                                    batch_first=True).to(DEVICE)
    output_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(output_ids)], \
                                                     batch_first=True).to(DEVICE)

    src_padding_mask = (input_ids_pad == PAD_IDX).to(DEVICE)
    tgt_padding_mask = (output_ids_pad == PAD_IDX).to(DEVICE)

    logits = model(input_ids_pad, output_ids_pad, src_padding_mask, tgt_padding_mask)

    pred = logits.argmax(2).item()

    while pred not in [tokenizer_en.token_to_id('[EOS]'), \
                       tokenizer_en.token_to_id('[PAD]')] and len(output_ids) < 100:
        output_ids.append(pred)
        output_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(output_ids)], \
                                                         batch_first=True).to(DEVICE)
        tgt_padding_mask = (output_ids_pad == PAD_IDX).to(DEVICE)

        logits = model(input_ids_pad, output_ids_pad, src_padding_mask, tgt_padding_mask)
        pred = logits.argmax(2).view(-1)[-1].item()

    return tokenizer_en.decoder.decode([tokenizer_en.id_to_token(i) for i in output_ids[1:]])

## Параметры для обучения

In [118]:
# попробуйте поставить параметры поменьше если в колабе обучается слишком долго!

vocab_size_enc = tokenizer_ru.get_vocab_size()
vocab_size_dec = tokenizer_en.get_vocab_size()

print(vocab_size_enc, vocab_size_dec)

embed_dim = 32 # еще называется D_MODEL
num_heads = 8
ff_dim = embed_dim * 2 # еще называется D_FF
num_layers = 4 # количество слоев

batch_size = 200
NUM_EPOCHS = 7

model = TransformerEncoderDecoder(vocab_size_enc,vocab_size_dec, embed_dim, num_heads, ff_dim, num_layers)

30000 30000


In [119]:
training_generator = torch.utils.data.DataLoader(training_set, batch_size=batch_size, shuffle=True, )
valid_generator = torch.utils.data.DataLoader(valid_set, batch_size=batch_size, shuffle=False)

In [120]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [121]:
model = model.to(DEVICE)
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX).to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001)

# подбирает learning rate в процессе обучения; бывают разные скедулеры
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.001, \
    pct_start=0.05, steps_per_epoch=len(training_generator), epochs=NUM_EPOCHS)

In [122]:
print(sum(p.numel() for p in model.parameters()) / 1e6, 'M parameters')

2.995632 M parameters


## Обучение

In [123]:
print(translate('Привет, мир!'))
print(translate('В Черноморске гремели крыши и по улицам гуляли сквозняки.!'))
print(translate('Что ты здесь забыл?'))
print(translate('Мама мыла раму.'))

obstacle Italian fish Italian fish CRsimplification fish presently Spiritfish 구fish ѓ sein double DID ECLAC Facilitation DID evidenced थstability messbriefing consolidate Spiritbriefing fish School double Spiritfish SpiritSinidentical M 242 You BARfederSpiritfish 𝓲fish Spirithomproliferation briefing SpiritDemographics 실fish के linen cclinen Spirittestified linen Spirit242 Recommended fish deterconsolidate 242 disbelieve Italian fish furthering trArifish 242 fish 𝓷 homM fish fish kee echanSpiritfish linen assi242 fish M trinvfish 242 threw fronts Spiritfish furthering
consolidate interventions fish fish fish fish conference 242 played fish fish messDemographics fish ½fish fish School fish Petersfish fish 242 BARfish fish axfish linen BARfish fish .). fish consolidate fish BAR242 fish 242 fish 242 briefing Petersinclude fish fish fish AriItalian fish fish fish fish xation obstacle fish promotional fish consolidate fish linen 242 linen sales consolidate fish stability consolidate fish fi

In [124]:
losses = []
run = None

for epoch in tqdm(range(1, NUM_EPOCHS + 1)):
    start_time = timer()
    train_loss = train(model, training_generator, optimizer, loss_fn, \
                       scheduler, run)
    # run.log({"epoch_loss": train_loss})
    end_time = timer()
    val_loss = evaluate(model, valid_generator, loss_fn, run)
    # run.log({"epoch_val_loss": val_loss})

    if not losses:
        print(f'First epoch - {val_loss}, saving model..')
        torch.save(model, 'model')

    elif val_loss < min(losses):
        print(f'Improved from {min(losses)} to {val_loss}, saving model..')
        torch.save(model, 'model')

    losses.append(val_loss)

    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, \
           "f"Epoch time={(end_time-start_time):.3f}s"))

    print(translate('Привет, мир!'))
    print(translate('В Черноморске гремели крыши и по улицам гуляли сквозняки.!'))
    print(translate('Что ты здесь забыл?'))
    print(translate('Мама мыла раму.'))

  0%|          | 0/7 [00:00<?, ?it/s]

Loss: 9.597413535118102;
Loss: 8.319493036270142;
Loss: 7.695869603951772;
Loss: 7.309091204404831;
First epoch - 5.8614000415802, saving model..
Epoch: 1, Train loss: 7.098, Val loss: 5.861,            Epoch time=108.230s
- You ' s !
You ' s a way .
What ' s you ' s ?


 14%|█▍        | 1/7 [01:50<11:02, 110.49s/it]

You ' s a .
Loss: 5.850856082439423;
Loss: 5.798836179971695;
Loss: 5.745991115570068;
Loss: 5.706532709598541;
Improved from 5.8614000415802 to 5.463237628936768, saving model..
Epoch: 2, Train loss: 5.681, Val loss: 5.463,            Epoch time=109.536s
Oh , sir !


 29%|██▊       | 2/7 [03:42<09:16, 111.33s/it]

...

The first of the same of the same of the same of the same of the same of the same of the same of the world , and the world of the world .
What are you ?
We ' re gonna be a lot .
Loss: 5.097927739620209;
Loss: 5.091773506402969;
Loss: 5.09265107234319;
Loss: 5.089913992881775;
Improved from 5.105348243713379 to 5.086424379348755, saving model..
Epoch: 6, Train loss: 5.090, Val loss: 5.086,            Epoch time=109.548s
Hey , hey !


 86%|████████▌ | 6/7 [11:10<01:51, 111.89s/it]

The first of the same of the same of the same of the same of the same of the same of the same of the world , and the world of the world .
What are you doing ?
- I ' m gonna be a lot .
Loss: 5.073873658180236;
Loss: 5.071031522750855;
Loss: 5.069715406099955;
Loss: 5.073418034911156;
Improved from 5.086424379348755 to 5.082032966613769, saving model..
Epoch: 7, Train loss: 5.072, Val loss: 5.082,            Epoch time=108.089s
Hey , hey !


100%|██████████| 7/7 [13:01<00:00, 111.63s/it]

The first of the same of the same of the same of the same of the same of the world , and the world of the world , and the world of the world .
What are you doing ?
- I ' m gonna be a lot .


## Оценка качества

In [125]:
to_translate = ru_sents[2000:2500]
ref_transl = en_sents[2000:2500]

In [132]:
ref_transl[0]

'Could you please stop staring?'

In [126]:
machine_transl = []
for ex in tqdm(to_translate):
  machine_transl.append(translate(ex))

100%|██████████| 500/500 [01:00<00:00,  8.27it/s]


In [127]:
machine_transl[0]

"We ' re gonna be a lot of the same ."

In [135]:
scores = dict()
for i in range(len(machine_transl)):
  reference = ref_transl[i].split()
  hypothesis = machine_transl[i].split()
  pair = (ref_transl[i], machine_transl[i])
  scores[pair] = (nltk.translate.bleu_score.sentence_bleu([reference], \
                         hypothesis, auto_reweigh=True))

/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [140]:
for i in sorted(scores.items(), key=lambda x: x[1], reverse=True)[:10]:
  print(*i)

('The time now is 04:27 PM .', 'The time now is 06 : 00 PM .') 0.3549481056010053
('Group of 77 (briefing by Mr. Pekka Patosaari, Coordinator and Head of the Sescretariat of the United Nations Forum on Forests; and environment and sustainable development issues)', '( b ) The United Nations of the United Nations of the United Nations and the United Nations and the United Nations of the United Nations and the United Nations of the United Nations and the United Nations and the United Nations of the United Nations and the United Nations and the United Nations') 0.05697859151313651
('What are you doing here?', 'What are you ?') 6.725854833444237e-78
('- What are you talking about?', 'What are you ?') 5.238101011110965e-78
('It is currently Mon May 16, 2016 2:37 pm', 'It is currently Sat May 22 , 2016 4 : 01 pm') 3.771272693679486e-78
('It is currently Wed May 18, 2016 4:12 pm', 'It is currently Sat May 22 , 2016 4 : 01 pm') 3.771272693679486e-78
('It should be noted that only the text kept 

Конечно, переводы плохие (хоть и не нулевые). Но это ожидаемо притом, что 1) я взял только часть данных, 2) я мало обучал.

# Задание 2 (2 балла).
Прочитайте главу про машинный перевод у Журафски и Маннига - https://web.stanford.edu/~jurafsky/slp3/13.pdf
Ответьте своими словами в чем заключается техника back translation? Для чего она применяется и что позволяет получить? Опишите по шагам как ее применить к паре en->ru на данных из семинара. Сколько моделей понадобится? Сколько запусков обучения нужно будет сделать?

Ответ должен содержать как минимум 10 предложений.

Суть техники backtranslation в том, что генерируются дополнительные данные для обучения модели (аугментация). Существуют и другие методы аугментации данных.

Применяется, когда в паре есть малоресурсный язык (т.е. для него мало данных). Позволяет увеличить обучающую выборку.

Допустим, что для русского данных больше, чем для английского, и мы хотим обучить модель en->ru.
- Мы должны обучить модель ru->en.
- С помощью обученной модели генерируем англ. тексты в пару к "лишним" русским.
- Добавляем в обучающую выборку эти псевдо-переводные, пары с "синтетическими" текстами (это не та же выборка, на которой обучали первую модель).
- Обучаем желаемую модель en->ru.

Понадобится две модели, каждую запускаем один раз (при условии, что мы довольны результатом).

В сеттинге backtranslation можно подбирать различные параметры модели (промежуточной, например). Кроме того, можно выбирать, какая доля обучающих примеров будет сгенерированной, можно включать несколько переводов для одной фразы и т.п.

Техника работает "удивительно хорошо".

(Jurafsky 2025: 17)